# Comparison of results
we need to take in the results from both (1) hypedsearch (or whatever its called) and (2) Neo-Fusion to see how they compare. Generated data is saved in the `experiment.json` files, so we can load that in to know the true value and compare that to what each get. The steps we should take are the following:
1. Load the `experiment.json` file
2. Load the `json` output from hypedsearch
3. Load the results of Neo-Fusion
4. See how well hypedsearch did at identifying both hybrid and non hybrid sequences
5. See how well Neo-Fusion did at identifying both hybrid and non hybrid sequences
6. Compare overlaps and non overlapping results

__NOTE__: all the data we are testing here are (1) IDEAL theoretical spectra, (2) the hybrids are only cis-spliced as Neo-Fusion cannot search for trans-spliced peptides

### Constants and imports

In [ ]:
import json

test_directory = '../data/testing_output/'
experiment_json_file = test_directory + 'experiment_info.json'


## 1. Load the `experiment.json` file

In [ ]:
with open(experiment_json_file, 'r') as o:
    exp = json.load(o)
    

## 2. Load `json` from hypedsearch

In [ ]:
summary = json.load(open(test_directory + 'summary.json', 'r'))


## 3. Load Neo-Fusion results